In [1]:
# TU LIMPIEZA Y ANÁLISIS AQUÍ

import pandas as pd
import seaborn as sns
import numpy as np

tv_df = pd.read_csv('tvs.csv')

tv_df


,_id,created_by,first_air_date,genres,id,in_production,last_air_date,name,number_of_episodes,number_of_seasons,...,original_language,original_name,overview,popularity,poster_path,production_companies,status,tagline,vote_average,vote_count
0,6457d26c7b901e08b141869d,[],1996-10-09,"[{""id"":35,""name"":""Comedy"",""_id"":{""$oid"":""6457d...",13,True,2022-02-16,How do you like Wednesday?,568.0,4,...,ja,水曜どうでしょう,How do you like Wednesday? was a Japanese tele...,21.207,/i2MwS6U0XzD8ad6aS3HiTNKz8ov.jpg,[],Returning Series,NaN,9.200,2
1,6457d26c7b901e08b141867a,"[{""id"":19303,""name"":""Kevin Smith"",""_id"":{""$oid...",2000-05-31,"[{""id"":16,""name"":""Animation"",""_id"":{""$oid"":""64...",2,False,2002-12-22,Clerks,6.0,1,...,en,Clerks,The continuing adventures of store clerks Dant...,10.620,/xunXvzFlkf1GGgMkCySA9CCFumB.jpg,"[{""id"":1558,""name"":""Touchstone Television"",""lo...",Canceled,NaN,6.897,78
2,6457d26c7b901e08b141869f,[],2006-08-20,"[{""id"":35,""name"":""Comedy"",""_id"":{""$oid"":""6457d...",14,False,2006-11-11,Shuriken School,26.0,1,...,en,Shuriken School,Shuriken School is a British animated series t...,3.034,/khXHvYKAvGqjtnKoLVYZr8r5yKD.jpg,"[{""id"":75257,""name"":""Zinkia Entertainment"",""lo...",Ended,NaN,5.500,6
3,6457d26c7b901e08b14186a5,"[{""id"":1074129,""name"":""Fred Rogers"",""_id"":{""$o...",1968-02-19,"[{""id"":10762,""name"":""Kids"",""_id"":{""$oid"":""6457...",15,False,2001-09-07,Mister Rogers' Neighborhood,850.0,31,...,en,Mister Rogers' Neighborhood,Mister Rogers' Neighborhood is an American chi...,19.167,/4Gz46uJg6MFaM4xeblbVNSxCyTr.jpg,"[{""id"":16021,""name"":""WQED"",""logo_path"":""/2bjAi...",Ended,Won't you be my neighbor?,4.977,88
4,6457d26c7b901e08b14186ab,"[{""id"":1211910,""name"":""Richard R. Lee"",""_id"":{...",NaN,[],16,False,2005-09-09,Sidewalks: Video Nite,399.0,11,...,en,Sidewalks: Video Nite,Sidewalks: Video Nite is an American televisio...,2.910,NaN,[],Ended,NaN,0.000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150797,645c00b1378e4a37f5092e21,[],2015-06-12,"[{""id"":99,""name"":""Documentary"",""_id"":{""$oid"":""...",226231,False,2015-06-13,北京之夜,4.0,1,...,zh,北京之夜,NaN,0.000,NaN,[],Ended,NaN,0.000,0
150798,645c00b1378e4a37f5092e23,[],2014-07-29,"[{""id"":99,""name"":""Documentary"",""_id"":{""$oid"":""...",226232,False,2014-09-02,北洋海军兴亡史,6.0,1,...,zh,北洋海军兴亡史,NaN,0.000,/rcNz0Amku0XWOC3LuLfRHiKYqS3.jpg,[],Ended,NaN,0.000,0
150799,645c00b1378e4a37f5092e25,[],2023-05-09,[],226233,True,2023-05-09,Turkey: Empire of Erdogan,1.0,1,...,en,Turkey: Empire of Erdogan,A few months after his brutal crackdown on pro...,0.000,/l7rcVP8Q5yY4M7O6rJSstV7eIin.jpg,[],Returning Series,NaN,0.000,0
150800,645c00b1378e4a37f5092e26,[],2023-01-24,[],226234,True,2023-01-24,Woooooo! Becoming Ric Flair,1.0,1,...,en,Woooooo! Becoming Ric Flair,Ric Flair Biography,0.000,NaN,[],Returning Series,NaN,0.000,0


In [2]:
tv_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150802 entries, 0 to 150801
Data columns (total 21 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   _id                   150802 non-null  object 
 1   created_by            150802 non-null  object 
 2   first_air_date        119932 non-null  object 
 3   genres                150802 non-null  object 
 4   id                    150802 non-null  int64  
 5   in_production         150802 non-null  bool   
 6   last_air_date         122108 non-null  object 
 7   name                  150796 non-null  object 
 8   number_of_episodes    150168 non-null  float64
 9   number_of_seasons     150802 non-null  int64  
 10  origin_country        150802 non-null  object 
 11  original_language     150802 non-null  object 
 12  original_name         150796 non-null  object 
 13  overview              84623 non-null   object 
 14  popularity            150802 non-null  float64
 15  

In [3]:
tv_df.isnull().sum()

_id                          0
created_by                   0
first_air_date           30870
genres                       0
id                           0
in_production                0
last_air_date            28694
name                         6
number_of_episodes         634
number_of_seasons            0
origin_country               0
original_language            0
original_name                6
overview                 66179
popularity                   0
poster_path              55875
production_companies         0
status                       0
tagline                 146238
vote_average                 0
vote_count                   0
dtype: int64

In [4]:
tv_df.isna().sum()

_id                          0
created_by                   0
first_air_date           30870
genres                       0
id                           0
in_production                0
last_air_date            28694
name                         6
number_of_episodes         634
number_of_seasons            0
origin_country               0
original_language            0
original_name                6
overview                 66179
popularity                   0
poster_path              55875
production_companies         0
status                       0
tagline                 146238
vote_average                 0
vote_count                   0
dtype: int64

In [5]:
tv_df.shape[0]

150802

In [6]:
import cleaning as cl

# Aplicar la función de limpieza a lo largo de las filas y crear nuevas columnas con datos limpios
tv_df["new_genres"] = tv_df.apply(cl.limpiar_genero, axis=1)
tv_df["new_creator"] = tv_df.apply(cl.limpiar_creador, axis=1)
tv_df["new_prductora"] = tv_df.apply(cl.limpiar_productora, axis=1)

#borrar columnas que no interesan
cl.borrar_columna(tv_df,"poster_path")
cl.borrar_columna(tv_df,"tagline")
cl.borrar_columna(tv_df,"overview")

#he considerado que las filas con name == NAN no deberian entrar en el análisis
cl.limpiar_filas_nan(tv_df, "name")
cl.limpiar_filas_nan(tv_df, "number_of_episodes")

#remplazar nans por valores más bonitos
cl.reemplazar_nan(tv_df, "number_of_episodes", "sin informacion")
cl.reemplazar_nan(tv_df, "first_air_date", "sin informacion")
cl.reemplazar_nan(tv_df, "last_air_date", "no ha terminado")


La columna 'poster_path' se ha eliminado del DataFrame.
La columna 'tagline' se ha eliminado del DataFrame.
La columna 'overview' se ha eliminado del DataFrame.


In [7]:
df_limpio = tv_df[["_id", "first_air_date", "id", "in_production", "last_air_date", "name", "number_of_episodes", "number_of_seasons","original_language", "popularity", "status", "vote_count", "new_genres", "new_creator", "new_prductora"]]

df_limpio.to_csv(r'clean_data.csv', index=False)

In [8]:
df_limpio.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150162 entries, 0 to 150801
Data columns (total 15 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   _id                 150162 non-null  object 
 1   first_air_date      150162 non-null  object 
 2   id                  150162 non-null  int64  
 3   in_production       150162 non-null  bool   
 4   last_air_date       150162 non-null  object 
 5   name                150162 non-null  object 
 6   number_of_episodes  150162 non-null  float64
 7   number_of_seasons   150162 non-null  int64  
 8   original_language   150162 non-null  object 
 9   popularity          150162 non-null  float64
 10  status              150162 non-null  object 
 11  vote_count          150162 non-null  int64  
 12  new_genres          150162 non-null  object 
 13  new_creator         150162 non-null  object 
 14  new_prductora       150162 non-null  object 
dtypes: bool(1), float64(2), int64(3), 

In [9]:
df_limpio.isna().sum()

_id                   0
first_air_date        0
id                    0
in_production         0
last_air_date         0
name                  0
number_of_episodes    0
number_of_seasons     0
original_language     0
popularity            0
status                0
vote_count            0
new_genres            0
new_creator           0
new_prductora         0
dtype: int64

### PREGUNTAS

In [10]:
#1 ¿Cual es la serie con la duración más corta?

df_limpio.number_of_episodes.value_counts()
#Hay muchos con 0 episodios. O 1 episodio


0.0       24304
1.0       21880
6.0        8514
8.0        6270
4.0        6019
          ...  
3269.0        1
1906.0        1
1434.0        1
3656.0        1
455.0         1
Name: number_of_episodes, Length: 933, dtype: int64

In [11]:
#2. ¿Cual es la serie con la duración más larga?

df_limpio['name'][df_limpio.number_of_episodes==df_limpio.number_of_episodes.max()]

82482    Tagesschau
Name: name, dtype: object

In [12]:
#3. ¿Cual es la serie con más temporadas? ¿Y la que tiene más capítulos?

df2 = df_limpio[["name", "number_of_episodes", "number_of_seasons"]][df_limpio.number_of_seasons==df_limpio.number_of_seasons.max()]
df2

,name,number_of_episodes,number_of_seasons
15769,Fuse,2518.0,345


In [13]:
#4. ¿Cual es el género más estrenado para cada mes y año?

In [14]:
#5. ¿Cual es la media de temporadas por género?

#para evitar tener series con demasiadas temporadas he decidido mirar el promedio 
#solamente de las que son de un genero especifico
df_sin_comas = df_limpio[~df_limpio['new_genres'].str.contains(',')]
media_temporadas_por_genero = df_sin_comas.groupby('new_genres')['number_of_seasons'].mean()
media_temporadas_por_genero.sort_values(ascending=False)

new_genres
Music                 4.289474
Talk                  4.188131
Reality               3.000000
Soap                  2.921671
News                  2.911111
Family                2.276018
Crime                 2.149789
Kids                  2.139048
Comedy                1.976452
Western               1.938931
Documentary           1.752753
Action & Adventure    1.439024
Animation             1.305151
Drama                 1.287857
Mystery               1.210909
Sci-Fi & Fantasy      1.197895
History               1.142857
War & Politics        1.118421
Sin género            1.021004
Romance               1.000000
Name: number_of_seasons, dtype: float64

In [15]:
#6. Listado de los 10 creadores más prolíficos.

creadores_prolificos = df_limpio[df_limpio['new_creator'] != 'Sin creador especificado'].value_counts('new_creator').head(11)
creadores_prolificos

new_creator
Khmerzoon                          105
Shotaro Ishinomori                  64
John de Mol                         62
Adrián Suar                         61
Simon Fuller                        48
Joseph Barbera, William Hanna       46
Yang Li-Hua                         44
Ekta Kapoor                         42
Mark Burnett                        36
R.J. Nuevas                         36
Carla Albuquerque, Beto Ribeiro     34
dtype: int64

In [16]:
#7. ¿Que género es el que más producen estos escritores?

In [17]:
#8. ¿Cuanto duran de media *en el aire* las series? ¿Y por género?

In [18]:
#tv_df[~tv_df['name'].notna()]

### TRANSFORMACIONES

In [20]:
#1. Crea una nueva columna llamada 'media_episodios' que el número de episodios por temporada de cada serie.
import numpy as np

df_limpio['media_episodios'] = df_limpio['number_of_episodes'] / type(df_limpio['number_of_seasons'])

#NO FUNCIONA PORQUE?!?!?!

TypeError: Cannot broadcast np.ndarray with operand of type <class 'type'>

In [21]:
# 2. Crea una nueva columna llamada 'habla_inglesa' que indique si la serie está en inglés o no.

df_limpio['habla_inglesa'] = (df_limpio['original_language'] == 'en')

#NO FUNCIONA PORQUE?!?!?!

/tmp/ipykernel_2331/732743746.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_limpio['habla_inglesa'] = (df_limpio['original_language'] == 'en')


In [22]:
#3. Crear una nueva columna llamada 'produccion_europea' que indique si la serie es de producción europea o no.

# Lista de códigos de idioma correspondientes a países europeos
codigos_europeos = ['en', 'fr', 'de', 'es', 'it', 'pt', 'nl', 'sv', 'da', 'fi', 'no', 'pl', 'ru', 'tr', 'el', 'hu']

# Crear la nueva columna 'produccion_europea'
df_limpio['produccion_europea'] = df_limpio['original_language'].isin(codigos_europeos)

#NO FUNCIONA PORQUE?!?!?!

/tmp/ipykernel_2331/2765054943.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_limpio['produccion_europea'] = df_limpio['original_language'].isin(codigos_europeos)


In [ ]:
#4. Crea una nueva columna llamada 'mes_cancelacion' que indique el mes en el que se canceló la serie.